In [1]:
import multimolecule  # you must import multimolecule to register models
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments

/home/prithvi/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def preprocess_function(examples):
    return tokenizer(examples["sequences"], truncation=True, padding=True)

In [3]:
df = pd.read_csv("dataset.csv")

In [4]:
df_stable = df[df["mfe_pre"]>df["mfe_mutant"]]
sequence = list(df_stable["sequence"])
index = list(df_stable["mutation_index"])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(sequence, index, test_size=0.2, random_state=42)

In [6]:
trainData = {
    "sequences": X_train,
    "label": y_train
}

In [7]:
testData = {
    "sequences": X_test,
    "label": y_test
}

In [8]:
trainDataset = Dataset.from_dict(trainData)
testDataset = Dataset.from_dict(testData)

In [9]:
modelName = 'multimolecule/ernierna.ss'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(modelName)

/home/prithvi/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
model = AutoModelForSequenceClassification.from_pretrained(modelName, num_labels=100)

Some weights of ErnieRnaForSequencePrediction were not initialized from the model checkpoint at multimolecule/ernierna.ss and are newly initialized: ['sequence_head.decoder.bias', 'sequence_head.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
tokenized_dataset_train = trainDataset.map(preprocess_function, batched=True)
tokenized_dataset_test = testDataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4326 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1082 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=192,
    warmup_steps=500,
    weight_decay=0.0001,
    logging_dir="./logs",
    logging_strategy="epoch",  # Log after each epoch
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",  # Save model after each epoch
    load_best_model_at_end=True,  # Load the best model when finished training (highest eval metric)
    metric_for_best_model="eval_loss",  # Use eval_loss to select the best model
    greater_is_better=False,  # Lower eval_loss is better
    report_to="none"  # Disable external logging services
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.606500,4.609120
2,4.605100,4.607323
3,4.602900,4.604926
4,4.599400,4.602423
5,4.594700,4.599637
6,4.589700,4.598314
7,4.581900,4.594956
8,4.572600,4.593126
9,4.562900,4.591609
10,4.550000,4.590164


TrainOutput(global_step=2300, training_loss=2.6919403117635974, metrics={'train_runtime': 914.1997, 'train_samples_per_second': 473.201, 'train_steps_per_second': 2.516, 'total_flos': 2.26954227913056e+16, 'train_loss': 2.6919403117635974, 'epoch': 100.0})

In [ ]:
history = trainer.state.log_history


In [ ]:
tokenizer("ACGU")

In [ ]:
import random
def generateRNA(length):
    bases = 'ACGU'
    return ''.join(random.choice(bases) for _ in range(length))

In [ ]:
inputSeq = generateRNA(100)

In [ ]:
import torch
def predict(seq):
    inputs = tokenizer(seq, return_tensors="pt").to("cuda:0")
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    idx = torch.argmax(probs, dim=-1).item()
    return idx

In [ ]:
for _ in range(100):
    inputSeq = generateRNA(100)
    print(predict(inputSeq))

In [13]:
128+64

192